RQ1: What are the frames used by CNN and FOX in the stories using candidates’ tweets as news sources during the 2016 and 2020 US presidential campaign?
-	Data analysis of frames in news stories (percent, other statistics) 
-	Comparison between the 2 networks, and also comparison between 2016 and 2020 (CNN2016-2020, and Fox2016-2020)

Create a table like this
             		CNN2016 	FOX2016 	CNN2020 	FOX2020
Frame 1			%	%		%		%
Frame 2
Frame 3
And so forth.....
Positive	-- include these 2, they are emotional frames
Negative 

Report differences, if any, CNN-FOX2016 and CNN-Fox2020, but also CNN2016-CNN2020, and FOX2016-FOX2020.


In [22]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [23]:
#open file
df=pd.read_csv('all.csv')
#cases STORYNUMBER 191 and 195 are removed since twitter had missingO values

In [24]:
#fixing coding errors
df['STORYEMOTIONALFRAMES'] = df['STORYEMOTIONALFRAMES'].replace(["3"], 'neutral')
df['STORYEMOTIONALFRAMES'] = df['STORYEMOTIONALFRAMES'].replace([3], 'neutral')
df['TWEETEMOTIONALFRAMES'] = df['TWEETEMOTIONALFRAMES'].replace(["3"], 'neutral')
df['TWEETEMOTIONALFRAMES'] = df['TWEETEMOTIONALFRAMES'].replace([3], 'neutral')
df['STORYCLAIMSANOTHERSOURCEISFALSE_01'] = df['STORYCLAIMSANOTHERSOURCEISFALSE'].replace(["yes"], 1)
df['STORYCLAIMSANOTHERSOURCEISFALSE_01'] = df['STORYCLAIMSANOTHERSOURCEISFALSE_01'].replace(["no"], 0)
df['STORYFACTCHECKSTWEET_01'] = df['STORYFACTCHECKSTWEET'].replace(["yes"], 1)
df['STORYFACTCHECKSTWEET_01'] = df['STORYFACTCHECKSTWEET_01'].replace(["no"], 0)
df['SENSATIONALISM_01'] = df['SENSATIONALISM'].replace(["yes"], 1)
df['SENSATIONALISM_01'] = df['SENSATIONALISM_01'].replace(["no"], 0)
#unit of analysis is tweet
dftweet=df[['STORYNUMBER', 'CODER', 'YEAR', 'PUBLICATIONDATE', 'URL', 'PUBLISHER', "PUBLISHERYEAR",
       'NUMBEROFTWEETSBYCANDIDATES', 'NUMBEROFTWEETSFROMOTHERS', 'TWEETBY',
       'TWEETPRESENTATION', 'TWEETGENERICFRAMESDOMINANT','TWEETEMOTIONALFRAMES',
       'TWEETCONTAINSFALSESTATEMENT', 'TWEETCLAIMSSOMETHINGELSEISFALSE',
       'TWEETTOPIC', 'TWEETROLETOTHESTORY', 'STORYROLETOTWEET', 'SRC_TOTAL', 'STORYFACTCHECKSTWEET', 'SENSATIONALISM', 'STORYGENERICFRAMESDOMINANT', 
       'STORYEMOTIONALFRAMES', 'STORYCLAIMSANOTHERSOURCEISFALSE_01', 'STORYFACTCHECKSTWEET_01', 'SENSATIONALISM_01']]
#unit of analysis is story
dfstory=df[['STORYNUMBER', 'CODER', 'YEAR', 'PUBLICATIONDATE', 'URL', 'PUBLISHER', "PUBLISHERYEAR",
       'STORYTYPE', 'STORYTOPIC', 'STORYGENERICFRAMESDOMINANT', 'STORYEMOTIONALFRAMES',
       'STORYCONTAINSFALSESTATEMENT', 'STORYCLAIMSANOTHERSOURCEISFALSE',
       'STORYFACTCHECKSTWEET', 'SENSATIONALISM', 'src_expert', 'src_opponent',
       'src_otherpolitician', 'src_privatecitizen', 'src_media', 'src_other', "TWEETROLETOTHESTORY", "STORYROLETOTWEET", "SRC_TOTAL", "TWEETBY",
       'STORYCLAIMSANOTHERSOURCEISFALSE_01', 'STORYFACTCHECKSTWEET_01', 'SENSATIONALISM_01']]
#remove duplicates
dfstory.drop_duplicates(subset=['STORYNUMBER'], keep='first', inplace=True)
#dv list for tweets and stories
DVtweet=['TWEETTOPIC','TWEETGENERICFRAMESDOMINANT', 'TWEETEMOTIONALFRAMES',
       #  'TWEETCONTAINSFALSESTATEMENT', 'TWEETCLAIMSSOMETHINGELSEISFALSE'
        ]
DVstory=['STORYTOPIC', 'STORYGENERICFRAMESDOMINANT', 'STORYEMOTIONALFRAMES',
       # 'STORYCONTAINSFALSESTATEMENT', 'STORYCLAIMSANOTHERSOURCEISFALSE'
       ]
DVnamesTweet=['the topic of the tweet','the generic frames of the tweets','the emotional frames of the tweets']
DVnamesStory=['the topic of the story','the generic frames of the stories','the emotional frames of the stories']
#list of different datasets
dftweet2020=dftweet.loc[dftweet['YEAR'] == 2020]
dftweet2016=dftweet.loc[dftweet['YEAR'] == 2016]
dftweetFOX=dftweet.loc[dftweet['PUBLISHER'] == "fox"]
dftweetCNN=dftweet.loc[dftweet['PUBLISHER'] == "cnn"]
dftweetFOX2016=dftweetFOX.loc[dftweetFOX['YEAR'] == 2016]
dftweetCNN2016=dftweetCNN.loc[dftweetCNN['YEAR'] == 2016]
dftweetFOX2020=dftweetFOX.loc[dftweetFOX['YEAR'] == 2020]
dftweetCNN2020=dftweetCNN.loc[dftweetCNN['YEAR'] == 2020]
dfstory2020=dfstory.loc[dfstory['YEAR'] == 2020]
dfstory2016=dfstory.loc[dfstory['YEAR'] == 2016]
dfstoryFOX=dfstory.loc[dfstory['PUBLISHER'] == "fox"]
dfstoryCNN=dfstory.loc[dfstory['PUBLISHER'] == "cnn"]
dfstoryFOX2016=dfstoryFOX.loc[dfstoryFOX['YEAR'] == 2016]
dfstoryCNN2016=dfstoryCNN.loc[dfstoryCNN['YEAR'] == 2016]
dfstoryFOX2020=dfstoryFOX.loc[dfstoryFOX['YEAR'] == 2020]
dfstoryCNN2020=dfstoryCNN.loc[dfstoryCNN['YEAR'] == 2020]
dftweetPres=dftweet.loc[dftweet['TWEETBY'].isin(['trump', 'biden'])]
dfstoryPres=dfstory.loc[dfstory['TWEETBY'].isin(['trump', 'biden'])]
dftweetPres2016=dftweetPres.loc[dftweetPres['YEAR'] == 2016]
dftweetPres2020=dftweetPres.loc[dftweetPres['YEAR'] == 2020]
dfstoryPres2016=dfstoryPres.loc[dfstoryPres['YEAR'] == 2016]
dfstoryPres2020=dfstoryPres.loc[dfstoryPres['YEAR'] == 2020]
dfstoryPresFOX=dfstoryPres.loc[dfstoryPres['PUBLISHER'] == "fox"]
dfstoryPresCNN=dfstoryPres.loc[dfstoryPres['PUBLISHER'] == "cnn"]
pubyear=['cnn2016','cnn2020','fox2016','fox2020']

In [25]:
rq1_1=pd.crosstab(dfstory['STORYGENERICFRAMESDOMINANT'], dfstory['PUBLISHERYEAR'], margins=True)# crosstab with raw numbers
rq1_1.reset_index(inplace=True)
rq1_1.columns.name = None# remove the name of the columns
rq1_1p=pd.crosstab(dfstory['STORYGENERICFRAMESDOMINANT'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100# crosstab with percentages
rq1_1p.reset_index(inplace=True)
rq1_1p.columns.name = None
rq1_1a=pd.merge(rq1_1, rq1_1p, on="STORYGENERICFRAMESDOMINANT")#merge the two tables
rq1_1a=rq1_1a[['STORYGENERICFRAMESDOMINANT', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]#reorder columns
rq1_1a["All_p"]=rq1_1a["All"]/rq1_1a.All.sum()*100
rq1_1a["All_p"]=rq1_1a["All_p"].round(2)
rq1_1a.loc["All"] = rq1_1a.sum()
# rq1_1a.to_csv('rq1_1a.csv')#export to csv
rq1_1a.set_index('STORYGENERICFRAMESDOMINANT', inplace=True)#set index
# print(rq1_1a)
rq1_2=pd.crosstab(dfstory['STORYEMOTIONALFRAMES'], dfstory['PUBLISHERYEAR'], margins=True)
rq1_2.reset_index(inplace=True)
rq1_2.columns.name = None
rq1_2p=pd.crosstab(dfstory['STORYEMOTIONALFRAMES'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq1_2p.reset_index(inplace=True)
rq1_2p.columns.name = None
rq1_2a=pd.merge(rq1_2, rq1_2p, on="STORYEMOTIONALFRAMES")
rq1_2a=rq1_2a[['STORYEMOTIONALFRAMES', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq1_2a["All_p"]=rq1_2a["All"]/rq1_2a.All.sum()*100
rq1_2a["All_p"]=rq1_2a["All_p"].round(2)
rq1_2a.loc["All"] = rq1_2a.sum()
# rq1_2a.to_csv('rq1_2a.csv')
# print(rq1_2a)
rq1a=pd.concat([rq1_1a, rq1_2a], axis=0)
rq1a.to_csv('rq1a.csv')
rq1a

,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p,STORYEMOTIONALFRAMES
conflict,55,85.94,91,76.47,15,83.33,56,73.68,217,78.34,NaN
economic,3,4.69,11,9.24,1,5.56,0,0.00,15,5.42,NaN
human interest,3,4.69,12,10.08,1,5.56,17,22.37,33,11.91,NaN
morality,1,1.56,0,0.00,1,5.56,0,0.00,2,0.72,NaN
responsibility,2,3.12,5,4.20,0,0.00,3,3.95,10,3.61,NaN
conflicteconomichuman interestmoralityresponsibility,64,100.00,119,99.99,18,100.01,76,100.00,277,100.00,NaN
0,60,93.75,108,90.76,16,84.21,55,72.37,239,85.97,negative
1,3,4.69,0,0.00,2,10.53,0,0.00,5,1.80,neutral
2,1,1.56,11,9.24,1,5.26,21,27.63,34,12.23,positive
All,64,100.00,119,100.00,19,100.00,76,100.00,278,100.00,negativeneutralpositive


In [26]:
print('STORYGENERICFRAMESDOMINANT')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYGENERICFRAMESDOMINANT'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYGENERICFRAMESDOMINANT'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYGENERICFRAMESDOMINANT'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYGENERICFRAMESDOMINANT'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
print("STORYEMOTIONALFRAMES")
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYEMOTIONALFRAMES'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYEMOTIONALFRAMES'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYEMOTIONALFRAMES'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYEMOTIONALFRAMES'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

STORYGENERICFRAMESDOMINANT
CNNvFox2016: chi2 = 1.54, p = 0.82
CNNvFox2020: chi2 = 11.79, p = 0.01
CNN2016v2020: chi2 = 5.06, p = 0.28
FOX2016v2020: chi2 = 11.48, p = 0.02
STORYEMOTIONALFRAMES
CNNvFox2016: chi2 = 1.81, p = 0.41
CNNvFox2020: chi2 = 10.13, p = 0.0
CNN2016v2020: chi2 = 9.36, p = 0.01
FOX2016v2020: chi2 = 11.57, p = 0.0


RQ2: What are the frames of the candidates’ tweets selected by journalists for inclusion in news stories?
-	Data analysis of frames in candidates tweets
-	Presentation of tweets in news stories 
-	2016 and 2020

Create a table like this
             	Trump2016	Pence	Hillary  Kaine Trump2020   Pence2020   Biden    Harris
Frame 1			%	%		%		%
Frame 2
Frame 3
And so forth.....
Positive	
Negative


In [27]:
rq2_1=pd.crosstab(dftweet['TWEETGENERICFRAMESDOMINANT'], dftweet['PUBLISHERYEAR'], margins=True)
rq2_1.reset_index(inplace=True)
rq2_1.columns.name = None
rq2_1p=pd.crosstab(dftweet['TWEETGENERICFRAMESDOMINANT'], dftweet['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq2_1p.reset_index(inplace=True)
rq2_1p.columns.name = None
rq2_1a=pd.merge(rq2_1, rq2_1p, on="TWEETGENERICFRAMESDOMINANT")
rq2_1a=rq2_1a[['TWEETGENERICFRAMESDOMINANT', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq2_1a["All_p"]=rq2_1a["All"]/rq2_1a.All.sum()*100
rq2_1a["All_p"]=rq2_1a["All_p"].round(2)
rq2_1a.loc["All"] = rq2_1a.sum()
# rq2_1a.to_csv('rq2_1a.csv')
rq2_1a
rq2_2=pd.crosstab(dftweet['TWEETEMOTIONALFRAMES'], dftweet['PUBLISHERYEAR'], margins=True)
rq2_2.reset_index(inplace=True)
rq2_2.columns.name = None
rq2_2p=pd.crosstab(dftweet['TWEETEMOTIONALFRAMES'], dftweet['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq2_2p.reset_index(inplace=True)
rq2_2p.columns.name = None
rq2_2a=pd.merge(rq2_2, rq2_2p, on="TWEETEMOTIONALFRAMES")
rq2_2a=rq2_2a[['TWEETEMOTIONALFRAMES', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq2_2a["All_p"]=rq2_2a["All"]/rq2_2a.All.sum()*100
rq2_2a["All_p"]=rq2_2a["All_p"].round(2)
rq2_2a.loc["All"] = rq2_2a.sum()
# rq2_2a.to_csv('rq2_2a.csv')
rq2_2a
rq2a=pd.concat([rq2_1a, rq2_2a], axis=0)
rq2a.to_csv('rq2a.csv')
rq2a

,TWEETGENERICFRAMESDOMINANT,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p,TWEETEMOTIONALFRAMES
0,conflict,94,76.42,73,51.77,20,58.82,73,60.83,260,62.20,NaN
1,economic,1,0.81,16,11.35,1,2.94,3,2.50,21,5.02,NaN
2,human interest,14,11.38,36,25.53,13,38.24,32,26.67,95,22.73,NaN
3,morality,4,3.25,9,6.38,0,0.00,3,2.50,16,3.83,NaN
4,responsibility,10,8.13,7,4.96,0,0.00,9,7.50,26,6.22,NaN
All,conflicteconomichuman interestmoralityresponsi...,123,99.99,141,99.99,34,100.00,120,100.00,418,100.00,NaN
0,NaN,104,84.55,84,59.15,20,58.82,72,60.00,280,66.83,negative
1,NaN,5,4.07,0,0.00,2,5.88,0,0.00,7,1.67,neutral
2,NaN,14,11.38,58,40.85,12,35.29,48,40.00,132,31.50,positive
All,NaN,123,100.00,142,100.00,34,99.99,120,100.00,419,100.00,negativeneutralpositive


In [28]:
print("TWEETGENERICFRAMESDOMINANT")
#CNNvFox2016
crosstab=pd.crosstab(dftweet2016['TWEETGENERICFRAMESDOMINANT'], dftweet2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dftweet2020['TWEETGENERICFRAMESDOMINANT'], dftweet2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dftweetCNN['TWEETGENERICFRAMESDOMINANT'], dftweetCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dftweetFOX['TWEETGENERICFRAMESDOMINANT'], dftweetFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
print("TWEETEMOTIONALFRAMES")
#CNNvFox2016
crosstab=pd.crosstab(dftweet2016['TWEETEMOTIONALFRAMES'], dftweet2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dftweet2020['TWEETEMOTIONALFRAMES'], dftweet2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dftweetCNN['TWEETEMOTIONALFRAMES'], dftweetCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dftweetFOX['TWEETEMOTIONALFRAMES'], dftweetFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

TWEETGENERICFRAMESDOMINANT
CNNvFox2016: chi2 = 17.12, p = 0.0
CNNvFox2020: chi2 = 10.76, p = 0.03
CNN2016v2020: chi2 = 26.91, p = 0.0
FOX2016v2020: chi2 = 4.65, p = 0.33
TWEETEMOTIONALFRAMES
CNNvFox2016: chi2 = 11.63, p = 0.0
CNNvFox2020: chi2 = 0.0, p = 0.99
CNN2016v2020: chi2 = 32.82, p = 0.0
FOX2016v2020: chi2 = 7.22, p = 0.03


RQ3: How did CNN.com and FoxNews.com report Donald Trump’s tweets compared to the other candidates in the 2016 and 2020 presidential election campaign news stories? Are there any significant differences between the two campaign coverages?
-	Report that the majority of news stories using tweets were using Trump’s tweets
-	Report presentation of tweets (embedding, citing, etc.)
-	Report the dominant frame of Trump’s tweets and the dominant frame in Hillary’s and Biden’s tweets but without repeating too much from RQ1 and RQ2
-	Report fact-checking, sensationalism, number of sources, and role of story to tweets for Trump vs Hillary and Biden. 


In [29]:
# Report that the majority of news stories using tweets were using Trump’s tweets
rq3_1=pd.crosstab(dfstoryPres['TWEETBY'], dfstoryPres['PUBLISHERYEAR'], margins=True)
rq3_1.reset_index(inplace=True)
rq3_1.columns.name = None
rq3_1p=pd.crosstab(dfstoryPres['TWEETBY'], dfstoryPres['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq3_1p.reset_index(inplace=True)
rq3_1p.columns.name = None
rq3_1a=pd.merge(rq3_1, rq3_1p, on="TWEETBY")
rq3_1a=rq3_1a[['TWEETBY', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq3_1a["All_p"]=rq3_1a["All"]/rq3_1a.All.sum()*100
rq3_1a["All_p"]=rq3_1a["All_p"].round(2)
rq3_1a.loc["All"] = rq3_1a.sum()
# rq3_1a.to_csv('rq3_1a.csv')
rq3_1a
# Report presentation of tweets (embedding, citing, etc.)
rq3_2=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['PUBLISHERYEAR'], margins=True)
rq3_2.reset_index(inplace=True)
rq3_2.columns.name = None
rq3_2p=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq3_2p.reset_index(inplace=True)
rq3_2p.columns.name = None
rq3_2a=pd.merge(rq3_2, rq3_2p, on="TWEETROLETOTHESTORY")
rq3_2a=rq3_2a[['TWEETROLETOTHESTORY', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq3_2a["All_p"]=rq3_2a["All"]/rq3_2a.All.sum()*100
rq3_2a["All_p"]=rq3_2a["All_p"].round(2)
rq3_2a.loc["All"] = rq3_2a.sum()
# rq3_2a.to_csv('rq3_2a.csv')

rq3_3=pd.crosstab(dfstory['STORYROLETOTWEET'], dfstory['PUBLISHERYEAR'], margins=True)
rq3_3.reset_index(inplace=True)
rq3_3.columns.name = None
rq3_3p=pd.crosstab(dfstory['STORYROLETOTWEET'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq3_3p.reset_index(inplace=True)
rq3_3p.columns.name = None
rq3_3a=pd.merge(rq3_3, rq3_3p, on="STORYROLETOTWEET")
rq3_3a=rq3_3a[['STORYROLETOTWEET', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq3_3a["All_p"]=rq3_3a["All"]/rq3_3a.All.sum()*100
rq3_3a["All_p"]=rq3_3a["All_p"].round(2)
rq3_3a.loc["All"] = rq3_3a.sum()
# rq3_3a.to_csv('rq3_3a.csv')
rq3_3a

rq3_1to3=pd.concat([rq3_1a, rq3_2a, rq3_3a], axis=0)
rq3_1to3.to_csv('rq3_1to3.csv')
rq3_1to3

,TWEETBY,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p,TWEETROLETOTHESTORY,STORYROLETOTWEET
0,biden,10,16.39,5,4.46,1,5.88,6,8.33,22,8.40,NaN,NaN
1,trump,51,83.61,107,95.54,16,94.12,66,91.67,240,91.60,NaN,NaN
All,bidentrump,61,100.00,112,100.00,17,100.00,72,100.00,262,100.00,NaN,NaN
0,NaN,34,53.12,36,30.25,8,42.11,24,31.58,102,36.69,illustration,NaN
1,NaN,15,23.44,33,27.73,9,47.37,3,3.95,60,21.58,non-primary source,NaN
2,NaN,15,23.44,50,42.02,2,10.53,49,64.47,116,41.73,trigger,NaN
All,NaN,64,100.00,119,100.00,19,100.01,76,100.00,278,100.00,illustrationnon-primary sourcetrigger,NaN
0,NaN,8,12.50,4,3.36,0,0.00,49,64.47,61,21.94,NaN,amplifies the tweet
1,NaN,34,53.12,59,49.58,1,5.26,4,5.26,98,35.25,NaN,ctiticizes the tweet
2,NaN,22,34.38,56,47.06,18,94.74,23,30.26,119,42.81,NaN,neutral


In [30]:
print('TWEETBY')
#CNNvFox2016
crosstab=pd.crosstab(dfstoryPres2016['TWEETBY'], dfstoryPres2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstoryPres2020['TWEETBY'], dfstoryPres2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryPresCNN['TWEETBY'], dfstoryPresCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryPresFOX['TWEETBY'], dfstoryPresFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

print('TWEETROLETOTHESTORY')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['TWEETROLETOTHESTORY'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['TWEETROLETOTHESTORY'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['TWEETROLETOTHESTORY'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['TWEETROLETOTHESTORY'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

print('STORYROLETOTWEET')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYROLETOTWEET'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYROLETOTWEET'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYROLETOTWEET'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYROLETOTWEET'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

TWEETBY
CNNvFox2016: chi2 = 0.5, p = 0.48
CNNvFox2020: chi2 = 0.58, p = 0.45
CNN2016v2020: chi2 = 5.67, p = 0.02
FOX2016v2020: chi2 = 0.0, p = 1.0
TWEETROLETOTHESTORY
CNNvFox2016: chi2 = 4.45, p = 0.11
CNNvFox2020: chi2 = 18.84, p = 0.0
CNN2016v2020: chi2 = 10.03, p = 0.01
FOX2016v2020: chi2 = 31.43, p = 0.0
STORYROLETOTWEET
CNNvFox2016: chi2 = 21.41, p = 0.0
CNNvFox2020: chi2 = 95.15, p = 0.0
CNN2016v2020: chi2 = 6.97, p = 0.03
FOX2016v2020: chi2 = 26.89, p = 0.0


In [31]:
# Report the dominant frame of Trump’s tweets and the dominant frame in Hillary’s and Biden’s tweets but without repeating too much from RQ1 and RQ2
# Report stories fact-checking tweets for Trump vs Hillary, 2016
rq3_a=pd.crosstab(dftweetPres2016['STORYGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'], margins=True)
rq3_a.reset_index(inplace=True)
rq3_a.columns.name = None
rq3_ap=pd.crosstab(dftweetPres2016['STORYGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_ap.reset_index(inplace=True)
rq3_ap.columns.name = None
rq3_aa=pd.merge(rq3_a, rq3_ap, on="STORYGENERICFRAMESDOMINANT")
rq3_aa=rq3_aa[['STORYGENERICFRAMESDOMINANT', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_aa["All_p"]=rq3_aa["All"]/rq3_aa.All.sum()*100
rq3_aa["All_p"]=rq3_aa["All_p"].round(2)
rq3_aa.loc["All"] = rq3_aa.sum()
# rq3_aa.to_csv('rq3_aa.csv')
rq3_aa

rq3_b=pd.crosstab(dftweetPres2016['TWEETGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'], margins=True)
rq3_b.reset_index(inplace=True)
rq3_b.columns.name = None
rq3_bp=pd.crosstab(dftweetPres2016['TWEETGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_bp.reset_index(inplace=True)
rq3_bp.columns.name = None
rq3_ba=pd.merge(rq3_b, rq3_bp, on="TWEETGENERICFRAMESDOMINANT")
rq3_ba=rq3_ba[['TWEETGENERICFRAMESDOMINANT', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_ba["All_p"]=rq3_ba["All"]/rq3_ba.All.sum()*100
rq3_ba["All_p"]=rq3_ba["All_p"].round(2)
rq3_ba.loc["All"] = rq3_ba.sum()
# rq3_ba.to_csv('rq3_ba.csv')
rq3_ba

rq3_c=pd.crosstab(dftweetPres2016['STORYEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], margins=True)
rq3_c.reset_index(inplace=True)
rq3_c.columns.name = None
rq3_cp=pd.crosstab(dftweetPres2016['STORYEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_cp.reset_index(inplace=True)
rq3_cp.columns.name = None
rq3_ca=pd.merge(rq3_c, rq3_cp, on="STORYEMOTIONALFRAMES")
rq3_ca=rq3_ca[['STORYEMOTIONALFRAMES', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_ca["All_p"]=rq3_ca["All"]/rq3_ca.All.sum()*100
rq3_ca["All_p"]=rq3_ca["All_p"].round(2)
rq3_ca.loc["All"] = rq3_ca.sum()
# rq3_ca.to_csv('rq3_ca.csv')
rq3_ca

rq3_d=pd.crosstab(dftweetPres2016['TWEETEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], margins=True)
rq3_d.reset_index(inplace=True)
rq3_d.columns.name = None
rq3_dp=pd.crosstab(dftweetPres2016['TWEETEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_dp.reset_index(inplace=True)
rq3_dp.columns.name = None
rq3_da=pd.merge(rq3_d, rq3_dp, on="TWEETEMOTIONALFRAMES")
rq3_da=rq3_da[['TWEETEMOTIONALFRAMES', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_da["All_p"]=rq3_da["All"]/rq3_da.All.sum()*100
rq3_da["All_p"]=rq3_da["All_p"].round(2)
rq3_da.loc["All"] = rq3_da.sum()
# rq3_da.to_csv('rq3_da.csv')
rq3_da

# Report stories fact-checking tweets for Trump vs Hillary, 2016
rq3_4=pd.crosstab(dftweetPres2016['STORYFACTCHECKSTWEET'], dftweetPres2016['TWEETBY'], margins=True)
rq3_4.reset_index(inplace=True)
rq3_4.columns.name = None
rq3_4p=pd.crosstab(dftweetPres2016['STORYFACTCHECKSTWEET'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_4p.reset_index(inplace=True)
rq3_4p.columns.name = None
rq3_4a=pd.merge(rq3_4, rq3_4p, on="STORYFACTCHECKSTWEET")
rq3_4a=rq3_4a[['STORYFACTCHECKSTWEET', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_4a["All_p"]=rq3_4a["All"]/rq3_4a.All.sum()*100
rq3_4a["All_p"]=rq3_4a["All_p"].round(2)
rq3_4a.loc["All"] = rq3_4a.sum()
# rq3_4a.to_csv('rq3_4a.csv')
rq3_4a

# Report stories sensationalism tweets for Trump vs Hillary, 2016
rq3_5=pd.crosstab(dftweetPres2016['SENSATIONALISM'], dftweetPres2016['TWEETBY'], margins=True)
rq3_5.reset_index(inplace=True)
rq3_5.columns.name = None
rq3_5p=pd.crosstab(dftweetPres2016['SENSATIONALISM'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_5p.reset_index(inplace=True)
rq3_5p.columns.name = None
rq3_5a=pd.merge(rq3_5, rq3_5p, on="SENSATIONALISM")
rq3_5a=rq3_5a[['SENSATIONALISM', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_5a["All_p"]=rq3_5a["All"]/rq3_5a.All.sum()*100
rq3_5a["All_p"]=rq3_5a["All_p"].round(2)
rq3_5a.loc["All"] = rq3_5a.sum()
# rq3_5a.to_csv('rq3_5a.csv')
rq3_5a

# Report stories tweet's role to stories for Trump vs Hillary, 2016
rq3_6=pd.crosstab(dftweetPres2016['TWEETROLETOTHESTORY'], dftweetPres2016['TWEETBY'], margins=True)
rq3_6.reset_index(inplace=True)
rq3_6.columns.name = None
rq3_6p=pd.crosstab(dftweetPres2016['TWEETROLETOTHESTORY'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_6p.reset_index(inplace=True)
rq3_6p.columns.name = None
rq3_6a=pd.merge(rq3_6, rq3_6p, on="TWEETROLETOTHESTORY")
rq3_6a=rq3_6a[['TWEETROLETOTHESTORY', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_6a["All_p"]=rq3_6a["All"]/rq3_6a.All.sum()*100
rq3_6a["All_p"]=rq3_6a["All_p"].round(2)
rq3_6a.loc["All"] = rq3_6a.sum()
# rq3_6a.to_csv('rq3_6a.csv')
rq3_6a

# Report stories story's role to tweets for Trump vs Hillary, 2016
rq3_7=pd.crosstab(dftweetPres2016['STORYROLETOTWEET'], dftweetPres2016['TWEETBY'], margins=True)
rq3_7.reset_index(inplace=True)
rq3_7.columns.name = None
rq3_7p=pd.crosstab(dftweetPres2016['STORYROLETOTWEET'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_7p.reset_index(inplace=True)
rq3_7p.columns.name = None
rq3_7a=pd.merge(rq3_7, rq3_7p, on="STORYROLETOTWEET")
rq3_7a=rq3_7a[['STORYROLETOTWEET', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_7a["All_p"]=rq3_7a["All"]/rq3_7a.All.sum()*100
rq3_7a["All_p"]=rq3_7a["All_p"].round(2)
rq3_7a.loc["All"] = rq3_7a.sum()
# rq3_7a.to_csv('rq3_7a.csv')
rq3_7a

rq3_4to7Hilary=pd.concat([rq3_aa, rq3_ba, rq3_ca, rq3_da, rq3_4a, rq3_5a, rq3_6a, rq3_7a], axis=0)
rq3_4to7Hilary.to_csv('rq3_4to7Hilary.csv')
rq3_4to7Hilary

,STORYGENERICFRAMESDOMINANT,trump_x,trump_y,biden_x,biden_y,All,All_p,TWEETGENERICFRAMESDOMINANT,STORYEMOTIONALFRAMES,TWEETEMOTIONALFRAMES,STORYFACTCHECKSTWEET,SENSATIONALISM,TWEETROLETOTHESTORY,STORYROLETOTWEET
0,conflict,117,93.60,11,57.89,128,88.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,economic,4,3.20,3,15.79,7,4.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,human interest,0,0.00,4,21.05,4,2.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,morality,1,0.80,1,5.26,2,1.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,responsibility,3,2.40,0,0.00,3,2.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
All,conflicteconomichuman interestmoralityresponsi...,125,100.00,19,99.99,144,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,103,78.63,8,42.11,111,74.00,conflict,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1,0.76,1,5.26,2,1.33,economic,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,16,12.21,7,36.84,23,15.33,human interest,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3,2.29,1,5.26,4,2.67,morality,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#rq3.2 comparing trump vs biden, 2020

rq3_a2=pd.crosstab(dftweetPres2020['STORYGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'], margins=True)
rq3_a2.reset_index(inplace=True)
rq3_a2.columns.name = None
rq3_a2p=pd.crosstab(dftweetPres2020['STORYGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_a2p.reset_index(inplace=True)
rq3_a2p.columns.name = None
rq3_a2a=pd.merge(rq3_a2, rq3_a2p, on="STORYGENERICFRAMESDOMINANT")
rq3_a2a=rq3_a2a[['STORYGENERICFRAMESDOMINANT', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_a2a["All_p"]=rq3_a2a["All"]/rq3_a2a.All.sum()*100
rq3_a2a["All_p"]=rq3_a2a["All_p"].round(2)
rq3_a2a.loc["All"] = rq3_a2a.sum()
# rq3_a2a.to_csv('rq3_a2a.csv')
rq3_a2a

rq3_b2=pd.crosstab(dftweetPres2020['TWEETGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'], margins=True)
rq3_b2.reset_index(inplace=True)
rq3_b2.columns.name = None
rq3_b2p=pd.crosstab(dftweetPres2020['TWEETGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_b2p.reset_index(inplace=True)
rq3_b2p.columns.name = None
rq3_b2a=pd.merge(rq3_b2, rq3_b2p, on="TWEETGENERICFRAMESDOMINANT")
rq3_b2a=rq3_b2a[['TWEETGENERICFRAMESDOMINANT', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_b2a["All_p"]=rq3_b2a["All"]/rq3_b2a.All.sum()*100
rq3_b2a["All_p"]=rq3_b2a["All_p"].round(2)
rq3_b2a.loc["All"] = rq3_b2a.sum()
# rq3_b2a.to_csv('rq3_b2a.csv')
rq3_b2a

rq3_c2=pd.crosstab(dftweetPres2020['STORYEMOTIONALFRAMES'], dftweetPres2020['TWEETBY'], margins=True)
rq3_c2.reset_index(inplace=True)
rq3_c2.columns.name = None
rq3_c2p=pd.crosstab(dftweetPres2020['STORYEMOTIONALFRAMES'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_c2p.reset_index(inplace=True)
rq3_c2p.columns.name = None
rq3_c2a=pd.merge(rq3_c2, rq3_c2p, on="STORYEMOTIONALFRAMES")
rq3_c2a=rq3_c2a[['STORYEMOTIONALFRAMES', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_c2a["All_p"]=rq3_c2a["All"]/rq3_c2a.All.sum()*100
rq3_c2a["All_p"]=rq3_c2a["All_p"].round(2)
rq3_c2a.loc["All"] = rq3_c2a.sum()
# rq3_c2a.to_csv('rq3_c2a.csv')
rq3_c2a

rq3_d2=pd.crosstab(dftweetPres2020['TWEETEMOTIONALFRAMES'], dftweetPres2020['TWEETBY'], margins=True)
rq3_d2.reset_index(inplace=True)
rq3_d2.columns.name = None
rq3_d2p=pd.crosstab(dftweetPres2020['TWEETEMOTIONALFRAMES'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_d2p.reset_index(inplace=True)
rq3_d2p.columns.name = None
rq3_d2a=pd.merge(rq3_d2, rq3_d2p, on="TWEETEMOTIONALFRAMES")
rq3_d2a=rq3_d2a[['TWEETEMOTIONALFRAMES', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_d2a["All_p"]=rq3_d2a["All"]/rq3_d2a.All.sum()*100
rq3_d2a["All_p"]=rq3_d2a["All_p"].round(2)
rq3_d2a.loc["All"] = rq3_d2a.sum()
# rq3_d2a.to_csv('rq3_d2a.csv')
rq3_d2a

# Report stories fact-checking tweets for Trump vs  vs  Biden, 2020
rq3_4_2=pd.crosstab(dftweetPres2020['STORYFACTCHECKSTWEET'], dftweetPres2020['TWEETBY'], margins=True)
rq3_4_2.reset_index(inplace=True)
rq3_4_2.columns.name = None
rq3_4_2p=pd.crosstab(dftweetPres2020['STORYFACTCHECKSTWEET'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_4_2p.reset_index(inplace=True)
rq3_4_2p.columns.name = None
rq3_4_2a=pd.merge(rq3_4_2, rq3_4_2p, on="STORYFACTCHECKSTWEET")
rq3_4_2a=rq3_4_2a[['STORYFACTCHECKSTWEET', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_4_2a["All_p"]=rq3_4_2a["All"]/rq3_4_2a.All.sum()*100
rq3_4_2a["All_p"]=rq3_4_2a["All_p"].round(2)
rq3_4_2a.loc["All"] = rq3_4_2a.sum()
# rq3_4_2a.to_csv('rq3_4_2a.csv')
rq3_4_2a

# Report stories sensationalism tweets for Trump vs  vs  Biden, 2020
rq3_5_2=pd.crosstab(dftweetPres2020['SENSATIONALISM'], dftweetPres2020['TWEETBY'], margins=True)
rq3_5_2.reset_index(inplace=True)
rq3_5_2.columns.name = None
rq3_5_2p=pd.crosstab(dftweetPres2020['SENSATIONALISM'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_5_2p.reset_index(inplace=True)
rq3_5_2p.columns.name = None
rq3_5_2a=pd.merge(rq3_5_2, rq3_5p, on="SENSATIONALISM")
rq3_5_2a=rq3_5_2a[['SENSATIONALISM', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_5_2a["All_p"]=rq3_5_2a["All"]/rq3_5_2a.All.sum()*100
rq3_5_2a["All_p"]=rq3_5_2a["All_p"].round(2)
rq3_5_2a.loc["All"] = rq3_5_2a.sum()
# rq3_5_2a.to_csv('rq3_5_2a.csv')
rq3_5_2a

# Report stories tweet's role to stories for Trump vs  vs  Biden, 2020
rq3_6_2=pd.crosstab(dftweetPres2020['TWEETROLETOTHESTORY'], dftweetPres2020['TWEETBY'], margins=True)
rq3_6_2.reset_index(inplace=True)
rq3_6_2.columns.name = None
rq3_6_2p=pd.crosstab(dftweetPres2020['TWEETROLETOTHESTORY'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_6_2p.reset_index(inplace=True)
rq3_6_2p.columns.name = None
rq3_6_2a=pd.merge(rq3_6_2, rq3_6_2p, on="TWEETROLETOTHESTORY")
rq3_6_2a=rq3_6_2a[['TWEETROLETOTHESTORY', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_6_2a["All_p"]=rq3_6_2a["All"]/rq3_6_2a.All.sum()*100
rq3_6_2a["All_p"]=rq3_6_2a["All_p"].round(2)
rq3_6_2a.loc["All"] = rq3_6_2a.sum()
# rq3_6_2a.to_csv('rq3_6_2a.csv')
rq3_6_2a

#report stories story's role to tweets for Trump vs  vs  Biden, 2020
rq3_7_2=pd.crosstab(dftweetPres2020['STORYROLETOTWEET'], dftweetPres2020['TWEETBY'], margins=True)
rq3_7_2.reset_index(inplace=True)
rq3_7_2.columns.name = None
rq3_7_2p=pd.crosstab(dftweetPres2020['STORYROLETOTWEET'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_7_2p.reset_index(inplace=True)
rq3_7_2p.columns.name = None
rq3_7_2a=pd.merge(rq3_7_2, rq3_7p, on="STORYROLETOTWEET")
rq3_7_2a=rq3_7_2a[['STORYROLETOTWEET', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_7_2a["All_p"]=rq3_7_2a["All"]/rq3_7_2a.All.sum()*100
rq3_7_2a["All_p"]=rq3_7_2a["All_p"].round(2)
rq3_7_2a.loc["All"] = rq3_7_2a.sum()
# rq3_7_2a.to_csv('rq3_7_2a.csv')
rq3_7_2a

rq3_4to7biden=pd.concat([rq3_a2a, rq3_b2a, rq3_c2a, rq3_d2a, rq3_4_2a, rq3_5_2a, rq3_6_2a, rq3_7_2a], axis=0)
rq3_4to7biden.to_csv('rq3_4to7biden.csv')
rq3_4to7biden

,STORYGENERICFRAMESDOMINANT,trump_x,trump_y,biden_x,biden_y,All,All_p,TWEETGENERICFRAMESDOMINANT,STORYEMOTIONALFRAMES,TWEETEMOTIONALFRAMES,STORYFACTCHECKSTWEET,SENSATIONALISM,TWEETROLETOTHESTORY,STORYROLETOTWEET
0,conflict,170,76.92,19,76.00,189,76.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,economic,13,5.88,0,0.00,13,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,human interest,32,14.48,4,16.00,36,14.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,responsibility,6,2.71,2,8.00,8,3.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
All,conflicteconomichuman interestresponsibility,221,99.99,25,100.00,246,99.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,132,60.00,12,48.00,144,58.78,conflict,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,17,7.73,1,4.00,18,7.35,economic,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,59,26.82,5,20.00,64,26.12,human interest,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4,1.82,4,16.00,8,3.27,morality,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,8,3.64,3,12.00,11,4.49,responsibility,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
#STORYGENERICFRAMESDOMINANT
print('2016')
crosstab=pd.crosstab(dftweetPres2016['STORYGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYGENERICFRAMESDOMINANT: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
#TWEETGENERICFRAMESDOMINANT
crosstab=pd.crosstab(dftweetPres2016['TWEETGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("TWEETGENERICFRAMESDOMINANT2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# STORYEMOTIONALFRAMES
crosstab=pd.crosstab(dftweetPres2016['STORYEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYEMOTIONALFRAMES2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# TWEETEMOTIONALFRAMES
crosstab=pd.crosstab(dftweetPres2016['TWEETEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("TWEETEMOTIONALFRAMES2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# STORYFACTCHECKSTWEET
crosstab=pd.crosstab(dftweetPres2016['STORYFACTCHECKSTWEET'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYFACTCHECKSTWEET2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# SENSATIONALISM
crosstab=pd.crosstab(dftweetPres2016['SENSATIONALISM'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("SENSATIONALISM2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# TWEETROLETOTHESTORY
crosstab=pd.crosstab(dftweetPres2016['TWEETROLETOTHESTORY'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("TWEETROLETOTHESTORY2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# STORYROLETOTWEET
crosstab=pd.crosstab(dftweetPres2016['STORYROLETOTWEET'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYROLETOTWEET2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
#STORYGENERICFRAMESDOMINANT
print('2020')
crosstab=pd.crosstab(dftweetPres2020['STORYGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYFACTCHECKSTWEET2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
#TWEETGENERICFRAMESDOMINANT
crosstab=pd.crosstab(dftweetPres2020['TWEETGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("TWEETGENERICFRAMESDOMINANT2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# STORYEMOTIONALFRAMES
crosstab=pd.crosstab(dftweetPres2020['STORYEMOTIONALFRAMES'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYEMOTIONALFRAMES2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# TWEETEMOTIONALFRAMES
crosstab=pd.crosstab(dftweetPres2020['TWEETEMOTIONALFRAMES'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("TWEETEMOTIONALFRAMES2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# STORYFACTCHECKSTWEET
crosstab=pd.crosstab(dftweetPres2020['STORYFACTCHECKSTWEET'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYFACTCHECKSTWEET2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# SENSATIONALISM
crosstab=pd.crosstab(dftweetPres2020['SENSATIONALISM'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("SENSATIONALISM2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# TWEETROLETOTHESTORY
crosstab=pd.crosstab(dftweetPres2020['TWEETROLETOTHESTORY'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("TWEETROLETOTHESTORY2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# STORYROLETOTWEET
crosstab=pd.crosstab(dftweetPres2020['STORYROLETOTWEET'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYROLETOTWEET2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

2016
STORYGENERICFRAMESDOMINANT: chi2 = 36.88, p = 0.0
TWEETGENERICFRAMESDOMINANT2016: chi2 = 13.11, p = 0.01
STORYEMOTIONALFRAMES2016: chi2 = 24.08, p = 0.0
TWEETEMOTIONALFRAMES2016: chi2 = 23.14, p = 0.0
STORYFACTCHECKSTWEET2016: chi2 = 4.93, p = 0.03
SENSATIONALISM2016: chi2 = 2.07, p = 0.15
TWEETROLETOTHESTORY2016: chi2 = 8.27, p = 0.02
STORYROLETOTWEET2016: chi2 = 14.49, p = 0.0
2020
STORYFACTCHECKSTWEET2020: chi2 = 3.44, p = 0.33
TWEETGENERICFRAMESDOMINANT2020: chi2 = 18.7, p = 0.0
STORYEMOTIONALFRAMES2020: chi2 = 0.04, p = 0.85
TWEETEMOTIONALFRAMES2020: chi2 = 4.62, p = 0.03
STORYFACTCHECKSTWEET2020: chi2 = 5.27, p = 0.02
SENSATIONALISM2020: chi2 = 11.57, p = 0.0
TWEETROLETOTHESTORY2020: chi2 = 0.56, p = 0.76
STORYROLETOTWEET2020: chi2 = 4.04, p = 0.13


In [34]:
pg.anova(data=dfstoryPres2020, dv='SRC_TOTAL', between=['TWEETBY','PUBLISHER'], detailed=True).round(2)
#cant be used: n for biden vs trump is too different

,Source,SS,DF,MS,F,p-unc,np2
0,TWEETBY,0.07,1.0,0.07,0.05,0.82,0.00
1,PUBLISHER,4.78,1.0,4.78,3.70,0.06,0.02
2,TWEETBY * PUBLISHER,0.05,1.0,0.05,0.04,0.84,0.00
3,Residual,232.29,180.0,1.29,NaN,NaN,NaN


In [35]:
pg.pairwise_ttests(data=dfstoryPres2020, dv='SRC_TOTAL', between=['TWEETBY','PUBLISHER'], padjust='bonf', effsize='cohen').round(2)
#cant be used: n for biden vs trump is too different

,Contrast,TWEETBY,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,TWEETBY,-,biden,trump,False,True,0.34,10.98,two-sided,0.74,NaN,NaN,0.319,0.12
1,PUBLISHER,-,cnn,fox,False,True,-1.98,157.24,two-sided,0.05,NaN,NaN,1.001,-0.30
2,TWEETBY * PUBLISHER,biden,cnn,fox,False,True,-0.24,8.87,two-sided,0.81,1.0,bonf,0.49,-0.15
3,TWEETBY * PUBLISHER,trump,cnn,fox,False,True,-1.97,145.09,two-sided,0.05,0.1,bonf,1.0,-0.30


RQ5: What journalistic values were used by CNN and FOX in the stories using candidates’ tweets as news stories?
-	Sensationalism.  (percent, other statistics, comparison between the 2 media organizations)
-	Fact-checking
-	Number of sources 
-	Role of story to tweets
-	Other values we coded for

In [36]:
rq5_1=pd.crosstab(dfstory['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_1.reset_index(inplace=True)
rq5_1.columns.name = None
rq5_1p=pd.crosstab(dfstory['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq5_1p.reset_index(inplace=True)
rq5_1p.columns.name = None
rq5_1a=pd.merge(rq5_1, rq5_1p, on="STORYCLAIMSANOTHERSOURCEISFALSE")
rq5_1a=rq5_1a[['STORYCLAIMSANOTHERSOURCEISFALSE', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq5_1a["All_p"]=rq5_1a["All"]/rq5_1a.All.sum()*100
rq5_1a["All_p"]=rq5_1a["All_p"].round(2)
rq5_1a.loc["All"] = rq5_1a.sum()
# rq5_1a.to_csv('rq5_1a.csv')
rq5_1a
rq5_2=pd.crosstab(dfstory['STORYFACTCHECKSTWEET'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_2.reset_index(inplace=True)
rq5_2.columns.name = None
rq5_2p=pd.crosstab(dfstory['STORYFACTCHECKSTWEET'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq5_2p.reset_index(inplace=True)
rq5_2p.columns.name = None
rq5_2a=pd.merge(rq5_2, rq5_2p, on="STORYFACTCHECKSTWEET")
rq5_2a=rq5_2a[['STORYFACTCHECKSTWEET', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq5_2a["All_p"]=rq5_2a["All"]/rq5_2a.All.sum()*100
rq5_2a["All_p"]=rq5_2a["All_p"].round(2)
rq5_2a.loc["All"] = rq5_2a.sum()
# rq5_2a.to_csv('rq5_2a.csv')
rq5_2a
rq5_3=pd.crosstab(dfstory['SENSATIONALISM'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_3.reset_index(inplace=True)
rq5_3.columns.name = None
rq5_3p=pd.crosstab(dfstory['SENSATIONALISM'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq5_3p.reset_index(inplace=True)
rq5_3p.columns.name = None
rq5_3a=pd.merge(rq5_3, rq5_3p, on="SENSATIONALISM")
rq5_3a=rq5_3a[['SENSATIONALISM', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq5_3a["All_p"]=rq5_3a["All"]/rq5_3a.All.sum()*100
rq5_3a["All_p"]=rq5_3a["All_p"].round(2)
rq5_3a.loc["All"] = rq5_3a.sum()
# rq5_3a.to_csv('rq5_3a.csv')
rq5_3a

rq5=pd.concat([rq5_1a, rq5_2a, rq5_3a], axis=0)
rq5.to_csv('rq5.csv')
rq5

,STORYCLAIMSANOTHERSOURCEISFALSE,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p,STORYFACTCHECKSTWEET,SENSATIONALISM
0,no,42,65.62,77,64.71,17,89.47,55,72.37,191,68.71,NaN,NaN
1,yes,22,34.38,42,35.29,2,10.53,21,27.63,87,31.29,NaN,NaN
All,noyes,64,100.00,119,100.00,19,100.00,76,100.00,278,100.00,NaN,NaN
0,NaN,43,67.19,80,67.23,18,94.74,68,89.47,209,75.18,no,NaN
1,NaN,21,32.81,39,32.77,1,5.26,8,10.53,69,24.82,yes,NaN
All,NaN,64,100.00,119,100.00,19,100.00,76,100.00,278,100.00,noyes,NaN
0,NaN,51,79.69,117,98.32,17,89.47,58,76.32,243,87.41,NaN,no
1,NaN,13,20.31,2,1.68,2,10.53,18,23.68,35,12.59,NaN,yes
All,NaN,64,100.00,119,100.00,19,100.00,76,100.00,278,100.00,NaN,noyes


In [37]:
print('STORYCLAIMSANOTHERSOURCEISFALSE')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
print('STORYFACTCHECKSTWEET')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYFACTCHECKSTWEET'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYFACTCHECKSTWEET'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYFACTCHECKSTWEET'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYFACTCHECKSTWEET'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
print('SENSATIONALISM')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['SENSATIONALISM'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['SENSATIONALISM'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['SENSATIONALISM'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['SENSATIONALISM'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

STORYCLAIMSANOTHERSOURCEISFALSE
CNNvFox2016: chi2 = 2.98, p = 0.08
CNNvFox2020: chi2 = 0.92, p = 0.34
CNN2016v2020: chi2 = 0.0, p = 1.0
FOX2016v2020: chi2 = 1.58, p = 0.21
STORYFACTCHECKSTWEET
CNNvFox2016: chi2 = 4.38, p = 0.04
CNNvFox2020: chi2 = 11.36, p = 0.0
CNN2016v2020: chi2 = 0.0, p = 1.0
FOX2016v2020: chi2 = 0.07, p = 0.79
SENSATIONALISM
CNNvFox2016: chi2 = 0.4, p = 0.53
CNNvFox2020: chi2 = 22.06, p = 0.0
CNN2016v2020: chi2 = 16.8, p = 0.0
FOX2016v2020: chi2 = 0.89, p = 0.35


RQ5 but in T-tests

In [38]:
dfstoryCNN2016.SRC_TOTAL.value_counts()

Series([], Name: count, dtype: int64)

In [39]:
#comparison of number of sources: Significant difference between CNN and FOX in 2020 (no data for 2016)
#variables
lista=[dfstoryCNN2020]
listb=[dfstoryFOX2020]
listc=['CNN in 2020']
listd=['FOX in 2020']
for i, j, k, l in zip(lista, listb, listc, listd):
    tresult=pg.ttest(i.SRC_TOTAL, j.SRC_TOTAL, paired=False).round(2)
    k='CNN'
    l='FOX'
    print('The difference in terms of '
            +i.SRC_TOTAL.name
            +' between '
            +k
            +' (m = '
            +i.SRC_TOTAL.mean().round(2).astype(str)
            +', sd = '
            +i.SRC_TOTAL.std().round(2).astype(str)
            +') and '
            +l
            +' (m = '
            +j.SRC_TOTAL.mean().round(2).astype(str)
            +', sd = '
            +i.SRC_TOTAL.std().round(2).astype(str)
            +') was '
            )
    if tresult.iloc[0,3] < 0.05:
        print('significant (T = '+tresult.iloc[0,0].astype(str)+', df = '+tresult.iloc[0,1].astype(str)+', p = '+tresult.iloc[0,3].astype(str)
            +", Cohen's d = "+tresult.iloc[0,5].astype(str)+').'
            )
    else:
        print('significant (T = '+tresult.iloc[0,0].astype(str)+', df = '+tresult.iloc[0,1].astype(str)+', p = '+tresult.iloc[0,3].astype(str)
            +", Cohen's d = "+tresult.iloc[0,5].astype(str)+').'
            )

The difference in terms of SRC_TOTAL between CNN (m = 1.63, sd = 1.14) and FOX (m = 1.96, sd = 1.14) was 
significant (T = -2.04, df = 166.75, p = 0.04, Cohen's d = 0.3).


In [40]:
#variables
lista=[dfstoryCNN2016, dfstoryCNN2016, dfstoryFOX2016, dfstoryCNN2020]
listb=[dfstoryFOX2016, dfstoryCNN2020, dfstoryFOX2020, dfstoryFOX2020]
listc=['CNN in 2016', 'CNN in 2016', 'FOX in 2016', 'CNN in 2020']
listd=['FOX in 2016', 'CNN in 2020', 'FOX in 2020', 'FOX in 2020']
#loop of t tests for STORYCLAIMSANOTHERSOURCEISFALSE_01
for i, j, k, l in zip(lista, listb, listc, listd):
    tresult=pg.ttest(i.STORYCLAIMSANOTHERSOURCEISFALSE_01, j.STORYCLAIMSANOTHERSOURCEISFALSE_01, paired=False).round(2)
#print result 
    print('The difference in terms of '
        +i.STORYCLAIMSANOTHERSOURCEISFALSE_01.name
        +' between '
        + k 
        +' (m = '
        +i.STORYCLAIMSANOTHERSOURCEISFALSE_01.mean().round(2).astype(str)
        +', sd = '
        +i.STORYCLAIMSANOTHERSOURCEISFALSE_01.std().round(2).astype(str)
        +') and '
        +l
        +' (m = '
        +j.STORYCLAIMSANOTHERSOURCEISFALSE_01.mean().round(2).astype(str)
        +', sd = '
        + i.STORYCLAIMSANOTHERSOURCEISFALSE_01.std().round(2).astype(str)
        +') was '
        )
    if tresult.iloc[0,3] < 0.05:
        print('significant (T = '
            +tresult.iloc[0,0].astype(str)
            +', df = '
            +tresult.iloc[0,1].astype(str)
            +', p = '
            +tresult.iloc[0,3].astype(str)
            +", Cohen's d = "
            +tresult.iloc[0,5].astype(str)
            +').'
            )
    else:
        print('not significant (T = '
            +tresult.iloc[0,0].astype(str)
            +', df = '
            +tresult.iloc[0,1].astype(str)
            +', p = '
            +tresult.iloc[0,3].astype(str)
            +", Cohen's d = "
            +tresult.iloc[0,5].astype(str)
            +').'
            )

The difference in terms of STORYCLAIMSANOTHERSOURCEISFALSE_01 between CNN in 2016 (m = 0.34, sd = 0.48) and FOX in 2016 (m = 0.11, sd = 0.48) was 
significant (T = 2.54, df = 45.04, p = 0.01, Cohen's d = 0.53).
The difference in terms of STORYCLAIMSANOTHERSOURCEISFALSE_01 between CNN in 2016 (m = 0.34, sd = 0.48) and CNN in 2020 (m = 0.35, sd = 0.48) was 
not significant (T = -0.12, df = 129.33, p = 0.9, Cohen's d = 0.02).
The difference in terms of STORYCLAIMSANOTHERSOURCEISFALSE_01 between FOX in 2016 (m = 0.11, sd = 0.32) and FOX in 2020 (m = 0.28, sd = 0.32) was 
not significant (T = -1.92, df = 38.61, p = 0.06, Cohen's d = 0.4).
The difference in terms of STORYCLAIMSANOTHERSOURCEISFALSE_01 between CNN in 2020 (m = 0.35, sd = 0.48) and FOX in 2020 (m = 0.28, sd = 0.48) was 
not significant (T = 1.13, df = 167.35, p = 0.26, Cohen's d = 0.16).


In [41]:
#variables
lista=[dfstoryCNN2016, dfstoryCNN2016, dfstoryFOX2016, dfstoryCNN2020]
listb=[dfstoryFOX2016, dfstoryCNN2020, dfstoryFOX2020, dfstoryFOX2020]
listc=['CNN in 2016', 'CNN in 2016', 'FOX in 2016', 'CNN in 2020']
listd=['FOX in 2016', 'CNN in 2020', 'FOX in 2020', 'FOX in 2020']
#loop of t tests for STORYFACTCHECKSTWEET_01
for i, j, k, l in zip(lista, listb, listc, listd):
    tresult=pg.ttest(i.STORYFACTCHECKSTWEET_01, j.STORYFACTCHECKSTWEET_01, paired=False).round(2)
#print result 
    print('The difference in terms of '
        +i.STORYFACTCHECKSTWEET_01.name
        +' between '
        + k 
        +' (m = '
        +i.STORYFACTCHECKSTWEET_01.mean().round(2).astype(str)
        +', sd = '
        +i.STORYFACTCHECKSTWEET_01.std().round(2).astype(str)
        +') and '
        +l
        +' (m = '
        +j.STORYFACTCHECKSTWEET_01.mean().round(2).astype(str)
        +', sd = '
        + i.STORYFACTCHECKSTWEET_01.std().round(2).astype(str)
        +') was '
        )
    if tresult.iloc[0,3] < 0.05:
        print('significant (T = '
            +tresult.iloc[0,0].astype(str)
            +', df = '
            +tresult.iloc[0,1].astype(str)
            +', p = '
            +tresult.iloc[0,3].astype(str)
            +", Cohen's d = "
            +tresult.iloc[0,5].astype(str)
            +').'
            )
    else:
        print('not significant (T = '
            +tresult.iloc[0,0].astype(str)
            +', df = '
            +tresult.iloc[0,1].astype(str)
            +', p = '
            +tresult.iloc[0,3].astype(str)
            +", Cohen's d = "
            +tresult.iloc[0,5].astype(str)
            +').'
            )

The difference in terms of STORYFACTCHECKSTWEET_01 between CNN in 2016 (m = 0.33, sd = 0.47) and FOX in 2016 (m = 0.05, sd = 0.47) was 
significant (T = 3.48, df = 63.33, p = 0.0, Cohen's d = 0.64).
The difference in terms of STORYFACTCHECKSTWEET_01 between CNN in 2016 (m = 0.33, sd = 0.47) and CNN in 2020 (m = 0.33, sd = 0.47) was 
not significant (T = 0.01, df = 128.62, p = 1.0, Cohen's d = 0.0).
The difference in terms of STORYFACTCHECKSTWEET_01 between FOX in 2016 (m = 0.05, sd = 0.23) and FOX in 2020 (m = 0.11, sd = 0.23) was 
not significant (T = -0.83, df = 36.23, p = 0.41, Cohen's d = 0.18).
The difference in terms of STORYFACTCHECKSTWEET_01 between CNN in 2020 (m = 0.33, sd = 0.47) and FOX in 2020 (m = 0.11, sd = 0.47) was 
significant (T = 3.98, df = 192.85, p = 0.0, Cohen's d = 0.53).


In [42]:
#variables
lista=[dfstoryCNN2016, dfstoryCNN2016, dfstoryFOX2016, dfstoryCNN2020]
listb=[dfstoryFOX2016, dfstoryCNN2020, dfstoryFOX2020, dfstoryFOX2020]
listc=['CNN in 2016', 'CNN in 2016', 'FOX in 2016', 'CNN in 2020']
listd=['FOX in 2016', 'CNN in 2020', 'FOX in 2020', 'FOX in 2020']
#loop of t tests for SENSATIONALISM_01
for i, j, k, l in zip(lista, listb, listc, listd):
    tresult=pg.ttest(i.SENSATIONALISM_01, j.SENSATIONALISM_01, paired=False).round(2)
#print result 
    print('The difference in terms of '
        +i.SENSATIONALISM_01.name
        +' between '
        + k 
        +'(m = '
        +i.SENSATIONALISM_01.mean().round(2).astype(str)
        +', sd = '
        +i.SENSATIONALISM_01.std().round(2).astype(str)
        +') and '
        +l
        +' (m = '
        +j.SENSATIONALISM_01.mean().round(2).astype(str)
        +', sd = '
        + i.SENSATIONALISM_01.std().round(2).astype(str)
        +') was '
        )
    if tresult.iloc[0,3] < 0.05:
        print('significant (T = '
            +tresult.iloc[0,0].astype(str)
            +', df = '
            +tresult.iloc[0,1].astype(str)
            +', p = '
            +tresult.iloc[0,3].astype(str)
            +", Cohen's d = "
            +tresult.iloc[0,5].astype(str)
            +').'
            )
    else:
        print('not significant (T = '
            +tresult.iloc[0,0].astype(str)
            +', df = '
            +tresult.iloc[0,1].astype(str)
            +', p = '
            +tresult.iloc[0,3].astype(str)
            +", Cohen's d = "
            +tresult.iloc[0,5].astype(str)
            +').'
            )

The difference in terms of SENSATIONALISM_01 between CNN in 2016(m = 0.2, sd = 0.41) and FOX in 2016 (m = 0.11, sd = 0.41) was 
not significant (T = 1.11, df = 37.44, p = 0.27, Cohen's d = 0.25).
The difference in terms of SENSATIONALISM_01 between CNN in 2016(m = 0.2, sd = 0.41) and CNN in 2020 (m = 0.02, sd = 0.41) was 
significant (T = 3.58, df = 69.94, p = 0.0, Cohen's d = 0.71).
The difference in terms of SENSATIONALISM_01 between FOX in 2016(m = 0.11, sd = 0.32) and FOX in 2020 (m = 0.24, sd = 0.32) was 
not significant (T = -1.51, df = 36.54, p = 0.14, Cohen's d = 0.32).
The difference in terms of SENSATIONALISM_01 between CNN in 2020(m = 0.02, sd = 0.13) and FOX in 2020 (m = 0.24, sd = 0.13) was 
significant (T = -4.36, df = 83.79, p = 0.0, Cohen's d = 0.77).
